In [1]:
import pandas as pd
import json
import elo

In [13]:
boards = [{1: [1, 2, 3, 4], 2: [4, 3, 2, 1]},
          {1: [1, 2, 3, 4], 2: [3, 4, 1, 2]},
          {1: [1, 2, 3, 4], 2: [2, 1, 4, 3]},
          {1: [1, 2, 3, 4], 2: [1, 2, 3, 4]}]

In [ ]:
date = '2018-02-07'

In [3]:
df = pd.DataFrame(
    [{'Day': date,
      'Round': rnd + 1,
      'Board offset': abs((rnd % 2 - 1) + (i // 2)),  # If board offset is even then player 1 is white
      'Team 1 division': division['division'],
      'Team 1 full name': f"{divisionList['name'].strip()} {divisionList['surname'].strip()}",
      'Board 1': boards[rnd][1][i],
      'Player 1': f"{player1['name'].strip()}",
      'Rating 1': int(f"{player1['rating'].strip()}"),
      'Team 2 division': division['division'],
      'Team 2 full name': f"{divisionList['name2'].strip()} {divisionList['surname2'].strip()}",
      'Board 2': boards[rnd][2][i],
      'Player 2': f"{player2['name'].strip()}",
      'Rating 2': int(f"{player2['rating'].strip()}")
     }
     for division in json.load(open(f'{date}.json'))
     for divisionList in division['divisionList']
     for rnd, rounds in enumerate(divisionList['rounds'])
     for i, (player1, player2) in enumerate(zip(rounds['team1'], rounds['team2']))])

In [4]:
# Anand is playing!
df[(df['Player 1'] == 'Viswanathan Anand') | (df['Player 2'] == 'Viswanathan Anand')]

,Board 1,Board 2,Board offset,Day,Player 1,Player 2,Rating 1,Rating 2,Round,Team 1 division,Team 1 full name,Team 2 division,Team 2 full name
16,1,4,1,2018-02-07,Viswanathan Anand,Johannes Haug,2794,2379,1,Eastern Division,Mumbai Movers,Eastern Division,Oslo Trolls
20,1,3,0,2018-02-07,Viswanathan Anand,Lars Oskar Hauge,2794,2433,2,Eastern Division,Mumbai Movers,Eastern Division,Oslo Trolls
24,1,2,1,2018-02-07,Viswanathan Anand,Frode Urkedal,2794,2570,3,Eastern Division,Mumbai Movers,Eastern Division,Oslo Trolls
28,1,1,0,2018-02-07,Viswanathan Anand,Benjamin Gledura,2794,2600,4,Eastern Division,Mumbai Movers,Eastern Division,Oslo Trolls


In [5]:
def score(x):
    # Player 1 is white on board 1 and 3 (board offset 0 and 2)
    p = x['Board offset'] % 2 == 0
    white_elo = x['Rating 1'] if p else x['Rating 2']
    black_elo = x['Rating 2'] if p else x['Rating 1']
    pr = elo.probabilities(white_elo, black_elo)
    return pd.Series({'Player 1 score': (pr['white'] if p else pr['black']) + pr['draw'] / 2,
                      'Player 2 score': (pr['black'] if p else pr['white']) + pr['draw'] / 2})

score_df = pd.concat([df, df.apply(score, axis=1)], axis=1)

In [6]:
# Sum score by player (+ day, division, board, team)
def f(i):
    return lambda x: pd.Series(
        {'Day': x['Day'],
         'Division': x[f'Team {i} division'],
         'Matchup': '{Team 1 full name} v {Team 2 full name}'.format(**x),
         'Board': x[f'Board {i}'],
         'Team': x[f'Team {i} full name'],
         'Player': x[f'Player {i}'],
         'Rating': x[f'Rating {i}'],
         'Score': x[f'Player {i} score']})    

total_df = pd.concat([score_df.apply(f(1), axis=1), score_df.apply(f(2), axis=1)]) \
    .groupby(['Day', 'Division', 'Matchup', 'Board', 'Team', 'Player']).agg({'Rating': 'mean', 'Score': 'sum'})

In [7]:
# Likely match scores
total_df.groupby(['Day', 'Division', 'Matchup', 'Team']).agg({'Rating': 'mean', 'Score': 'sum'})

Rating  \
Day        Division          Matchup                                            Team                                
2018-02-07 Atlantic Division Buenos Aires Krakens v Montclair Sopranos          Buenos Aires Krakens      2460.50   
                                                                                Montclair Sopranos        2494.75   
                             Montreal Chessbrahs v Minnesota Blizzard           Minnesota Blizzard        2472.25   
                                                                                Montreal Chessbrahs       2498.75   
                             Saint Louis Arch Bishops v Pittsburgh Pawngrabbers Pittsburgh Pawngrabbers   2468.50   
                                                                                Saint Louis Arch Bishops  2519.00   
                             Webster Windmills v Miami Champions                Miami Champions           2413.50   
                                                                                Webster Windmills         2500.75   
           Central Division  Cannes Blockbusters v London Lions                 Cannes Blockbusters       2500.00   
                                                                                London Lions              2488.50   
                             London Towers v Amsterdam Mosquitoes               Amsterdam Mosquitoes      2493.50   
                                                                                London Towers             2423.00   
                             Marseille Migraines v Ljubljana Turtles            Ljubljana Turtles         2497.75   
                                                                                Marseille Migraines       2515.25   
                             Stockholm Snowballs v Reykjavik Puffins            Reykjavik Puffins         2466.25   
                                                                                Stockholm Snowballs       2510.75   
           Eastern Division  Armenia Eagles v Volga Stormbringers               Armenia Eagles            2481.25   
                                                                                Volga Stormbringers       2493.50   
                             Estonia Horses v Delhi Dynamite                    Delhi Dynamite            2492.50   
                                                                                Estonia Horses            2436.50   
                             Mumbai Movers v Oslo Trolls                        Mumbai Movers             2497.75   
                                                                                Oslo Trolls               2495.50   
                             Norway Gnomes v Riga Magicians                     Norway Gnomes             2496.75   
                                                                                Riga Magicians            2505.50   
           Pacific Division  Chengdu Pandas v Australia Kangaroos               Australia Kangaroos       2459.00   
                                                                                Chengdu Pandas            2535.50   
                             San Diego Surfers v Rio Grande Ospreys             Rio Grande Ospreys        2497.25   
                                                                                San Diego Surfers         2489.50   
                             San Jose Hackers v Las Vegas Desert Rats           Las Vegas Desert Rats     2473.50   
                                                                                San Jose Hackers          2516.75   
                             Seattle Sluggers v Dallas Destiny                  Dallas Destiny            2495.50   
                                                                                Seattle Sluggers          2511.50   

                                                                                                             Score  
Day        

In [8]:
# Top scoring players per board
total_df.groupby(level=['Day', 'Division', 'Board'], group_keys=False).apply(lambda x: x.sort_values('Score', ascending=False).head(1))

Rating  \
Day        Division          Matchup                                            Board Team                     Player                           
2018-02-07 Atlantic Division Saint Louis Arch Bishops v Pittsburgh Pawngrabbers 1     Saint Louis Arch Bishops Fabiano Caruana           2799   
                                                                                2     Saint Louis Arch Bishops Vladimir Fedoseev         2731   
                                                                                3     Saint Louis Arch Bishops Alejandro Ramirez         2563   
                             Montreal Chessbrahs v Minnesota Blizzard           4     Minnesota Blizzard       Thomas Beerdsen           2428   
           Central Division  Marseille Migraines v Ljubljana Turtles            1     Marseille Migraines      Maxime Vachier-Lagrave    2804   
                                                                                2     Marseille Migraines      Etienne Bacrot            2728   
                                                                                3     Marseille Migraines      Yannick Gozzoli           2582   
                             Stockholm Snowballs v Reykjavik Puffins            4     Reykjavik Puffins        Bjorn Thorfinnsson        2398   
           Eastern Division  Mumbai Movers v Oslo Trolls                        1     Mumbai Movers            Viswanathan Anand         2794   
                             Norway Gnomes v Riga Magicians                     2     Riga Magicians           Arturs Neiksans           2584   
                             Armenia Eagles v Volga Stormbringers               3     Armenia Eagles           Manuel Petrosyan          2559   
                             Estonia Horses v Delhi Dynamite                    4     Delhi Dynamite           Hemant Sharma             2342   
           Pacific Division  San Jose Hackers v Las Vegas Desert Rats           1     San Jose Hackers         Shakhriyar Mamedyarov     2797   
                                                                                2     San Jose Hackers         Daniel Naroditsky         2626   
                             Chengdu Pandas v Australia Kangaroos               3     Chengdu Pandas           Xu Xiangyu                2549   
                             Seattle Sluggers v Dallas Destiny                  4     Dallas Destiny           Titas Stremavicius        2426   

                                                                                                                                          Score  
Day        Division          Matchup                                            Board Team                     Player                            
2018-02-07 Atlantic Division Saint Louis Arch Bishops v Pittsburgh Pawngrabbers 1     Saint Louis Arch Bishops Fabiano Caruana         3.463578  
                                                                                2     Saint Louis Arch Bishops Vladimir Fedoseev       3.233725  
                                                                                3     Saint Louis Arch Bishops Alejandro Ramirez       2.489152  
                             Montreal Chessbrahs v Minnesota Blizzard           4     Minnesota Blizzard       Thomas Beerdsen         1.640383  
           Central Division  Marseille Migraines v Ljubljana Turtles            1     Marseille Migraines      Maxime Vachier-Lagrave  3.391578  
                                                                                2     Marseille Migraines      Etienne Bacrot          3.092037  
                                                                                3     Marseille Migraines      Yannick Gozzoli         2.410209  
                             Stockholm Snowballs v Reykjavik Puffins            4     Reykjavik Puffins        Bjorn Thorfinnsson      1.427868  
           Eastern Division  Mumbai Movers v Oslo Trolls        

In [9]:
# Top players by rating
total_df.groupby(level=['Day', 'Division', 'Board'], group_keys=False).apply(lambda x: x.sort_values('Rating', ascending=False).head(1))

Rating  \
Day        Division          Matchup                                            Board Team                     Player                            
2018-02-07 Atlantic Division Saint Louis Arch Bishops v Pittsburgh Pawngrabbers 1     Saint Louis Arch Bishops Fabiano Caruana            2799   
                                                                                2     Saint Louis Arch Bishops Vladimir Fedoseev          2731   
                                                                                3     Saint Louis Arch Bishops Alejandro Ramirez          2563   
                             Montreal Chessbrahs v Minnesota Blizzard           4     Minnesota Blizzard       Thomas Beerdsen            2428   
           Central Division  Marseille Migraines v Ljubljana Turtles            1     Marseille Migraines      Maxime Vachier-Lagrave     2804   
                                                                                2     Marseille Migraines      Etienne Bacrot             2728   
                                                                                3     Marseille Migraines      Yannick Gozzoli            2582   
                             Stockholm Snowballs v Reykjavik Puffins            4     Reykjavik Puffins        Bjorn Thorfinnsson         2398   
           Eastern Division  Mumbai Movers v Oslo Trolls                        1     Mumbai Movers            Viswanathan Anand          2794   
                             Norway Gnomes v Riga Magicians                     2     Riga Magicians           Arturs Neiksans            2584   
                             Armenia Eagles v Volga Stormbringers               3     Armenia Eagles           Manuel Petrosyan           2559   
                             Norway Gnomes v Riga Magicians                     4     Norway Gnomes            Torbjørn Ringdal Hansen    2397   
           Pacific Division  San Jose Hackers v Las Vegas Desert Rats           1     San Jose Hackers         Shakhriyar Mamedyarov      2797   
                                                                                2     San Jose Hackers         Daniel Naroditsky          2626   
                             Chengdu Pandas v Australia Kangaroos               3     Chengdu Pandas           Xu Xiangyu                 2549   
                             San Jose Hackers v Las Vegas Desert Rats           4     Las Vegas Desert Rats    Jacek Stopa                2428   

                                                                                                                                           Score  
Day        Division          Matchup                                            Board Team                     Player                             
2018-02-07 Atlantic Division Saint Louis Arch Bishops v Pittsburgh Pawngrabbers 1     Saint Louis Arch Bishops Fabiano Caruana          3.463578  
                                                                                2     Saint Louis Arch Bishops Vladimir Fedoseev        3.233725  
                                                                                3     Saint Louis Arch Bishops Alejandro Ramirez        2.489152  
                             Montreal Chessbrahs v Minnesota Blizzard           4     Minnesota Blizzard       Thomas Beerdsen          1.640383  
           Central Division  Marseille Migraines v Ljubljana Turtles            1     Marseille Migraines      Maxime Vachier-Lagrave   3.391578  
                                                                                2     Marseille Migraines      Etienne Bacrot           3.092037  
                                                                                3     Marseille Migraines      Yannick Gozzoli          2.410209  
                             Stockholm Snowballs v Reykjavik Puffins            4     Reykjavik Puffins        Bjorn Thorfinnsson       1.427868  
           Eastern Division  Mumbai M

In [10]:
# Tie breaker
total_df.groupby(['Day', 'Team']).agg({'Rating': 'mean', 'Score': 'sum'}).sort_values('Score', ascending=False)

Rating     Score
Day        Team                                       
2018-02-07 Chengdu Pandas            2535.50  9.466305
           Saint Louis Arch Bishops  2519.00  9.393340
           Amsterdam Mosquitoes      2493.50  9.332219
           Webster Windmills         2500.75  9.282322
           Marseille Migraines       2515.25  9.027976
           Delhi Dynamite            2492.50  9.015001
           Stockholm Snowballs       2510.75  8.948100
           Montclair Sopranos        2494.75  8.721329
           San Jose Hackers          2516.75  8.719372
           Montreal Chessbrahs       2498.75  8.527884
           Cannes Blockbusters       2500.00  8.261970
           Seattle Sluggers          2511.50  8.246138
           Riga Magicians            2505.50  8.243755
           Rio Grande Ospreys        2497.25  8.213800
           Volga Stormbringers       2493.50  8.128234
           Oslo Trolls               2495.50  8.048437
           Mumbai Movers             2497.75  7.951563
           Armenia Eagles            2481.25  7.871766
           San Diego Surfers         2489.50  7.786200
           Norway Gnomes             2496.75  7.756245
           Dallas Destiny            2495.50  7.753862
           London Lions              2488.50  7.738030
           Minnesota Blizzard        2472.25  7.472116
           Las Vegas Desert Rats     2473.50  7.280628
           Buenos Aires Krakens      2460.50  7.278671
           Reykjavik Puffins         2466.25  7.051900
           Estonia Horses            2436.50  6.984999
           Ljubljana Turtles         2497.75  6.972024
           Miami Champions           2413.50  6.717678
           London Towers             2423.00  6.667781
           Pittsburgh Pawngrabbers   2468.50  6.606660
           Australia Kangaroos       2459.00  6.533695

In [11]:
# Example matchup
total_df[total_df.index.get_level_values('Matchup') == 'Chengdu Pandas v Australia Kangaroos']

Rating  \
Day        Division         Matchup                              Board Team                Player                     
2018-02-07 Pacific Division Chengdu Pandas v Australia Kangaroos 1     Australia Kangaroos Alexey Sarana       2541   
                                                                       Chengdu Pandas      Wang Yue            2699   
                                                                 2     Australia Kangaroos Anton Smirnov       2508   
                                                                       Chengdu Pandas      Ju Wenjun           2574   
                                                                 3     Australia Kangaroos Temur Kuybokarov    2468   
                                                                       Chengdu Pandas      Xu Xiangyu          2549   
                                                                 4     Australia Kangaroos Brandon Clarke      2319   
                                                                       Chengdu Pandas      Zhai Mo             2320   

                                                                                                                Score  
Day        Division         Matchup                              Board Team                Player                      
2018-02-07 Pacific Division Chengdu Pandas v Australia Kangaroos 1     Australia Kangaroos Alexey Sarana     2.044651  
                                                                       Chengdu Pandas      Wang Yue          3.166906  
                                                                 2     Australia Kangaroos Anton Smirnov     1.824068  
                                                                       Chengdu Pandas      Ju Wenjun         2.578592  
                                                                 3     Australia Kangaroos Temur Kuybokarov  1.668605  
                                                                       Chengdu Pandas      Xu Xiangyu        2.452480  
                                                                 4     Australia Kangaroos Brandon Clarke    0.996370  
                                                                       Chengdu Pandas      Zhai Mo           1.268327

In [12]:
score_df[(score_df['Team 1 full name'] == 'Chengdu Pandas') | (score_df['Team 2 full name'] == 'Chengdu Pandas')]

,Board 1,Board 2,Board offset,Day,Player 1,Player 2,Rating 1,Rating 2,Round,Team 1 division,Team 1 full name,Team 2 division,Team 2 full name,Player 1 score,Player 2 score
240,1,4,1,2018-02-07,Wang Yue,Brandon Clarke,2699,2319,1,Pacific Division,Chengdu Pandas,Pacific Division,Australia Kangaroos,0.901735,0.098265
241,2,3,1,2018-02-07,Ju Wenjun,Temur Kuybokarov,2574,2468,1,Pacific Division,Chengdu Pandas,Pacific Division,Australia Kangaroos,0.618293,0.381707
242,3,2,0,2018-02-07,Xu Xiangyu,Anton Smirnov,2549,2508,1,Pacific Division,Chengdu Pandas,Pacific Division,Australia Kangaroos,0.588621,0.411379
243,4,1,0,2018-02-07,Zhai Mo,Alexey Sarana,2320,2541,1,Pacific Division,Chengdu Pandas,Pacific Division,Australia Kangaroos,0.235455,0.764545
244,1,3,0,2018-02-07,Wang Yue,Temur Kuybokarov,2699,2468,2,Pacific Division,Chengdu Pandas,Pacific Division,Australia Kangaroos,0.805381,0.194619
245,2,4,0,2018-02-07,Ju Wenjun,Brandon Clarke,2574,2319,2,Pacific Division,Chengdu Pandas,Pacific Division,Australia Kangaroos,0.826776,0.173224
246,3,1,1,2018-02-07,Xu Xiangyu,Alexey Sarana,2549,2541,2,Pacific Division,Chengdu Pandas,Pacific Division,Australia Kangaroos,0.476277,0.523723
247,4,2,1,2018-02-07,Zhai Mo,Anton Smirnov,2320,2508,2,Pacific Division,Chengdu Pandas,Pacific Division,Australia Kangaroos,0.237614,0.762386
248,1,2,1,2018-02-07,Wang Yue,Anton Smirnov,2699,2508,3,Pacific Division,Chengdu Pandas,Pacific Division,Australia Kangaroos,0.728738,0.271262
249,2,1,1,2018-02-07,Ju Wenjun,Alexey Sarana,2574,2541,3,Pacific Division,Chengdu Pandas,Pacific Division,Australia Kangaroos,0.512565,0.487435
